In [1]:
# Libraries used
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from datetime import time

In [2]:
# Then read the CSV and create the new column
df = pd.read_csv('eurocup.csv', encoding='latin-1')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.columns = df.columns.str.lower()

In [3]:
# First, remove all spaces from the column
df['playtype'] = df['playtype'].str.strip()

# Create a dictionary for the mapping
mapping = {
    'LAYUPMD': '2FGM',
    'DUNK': '2FGM',
    'LAYUPATT': '2FGA',
    '2FGAB': '2FGA',
    '3FGAB': '3FGA'
}

# Apply the mapping
df['playtype'] = df['playtype'].replace(mapping)

In [6]:
# Step 1: Normalize "markertime" to a string format if it contains `datetime.time` objects
df['markertime'] = df['markertime'].apply(lambda x: x.strftime('%M:%S') if isinstance(x, pd._libs.tslibs.nattype.NaTType) else str(x))

# Step 2: Convert the "markertime" column to a timedelta format
df['markertime'] = pd.to_timedelta(df['markertime'], errors='coerce')

# Step 3: Identify games where "markertime" exceeds 00:10:00
invalid_games = df.loc[df['markertime'] > pd.Timedelta(minutes=10), 'game_id'].unique()

# Step 4: Filter out these invalid games from the dataframe
df = df[~df['game_id'].isin(invalid_games)]


In [7]:
def assign_quarter(minute):
    if minute < 11:
        return 1
    elif minute < 21:
        return 2
    elif minute < 31:
        return 3
    else:
        return 4

# Create new quarter column
df['quarter'] = df['minute'].apply(assign_quarter)

In [9]:
# Store initial count
initial_rows = len(df)

# List of playtypes to remove
playtypes_to_remove = ['EP', 'EG', 'BP', 'TPOFF', 'JB']

# Remove rows where playtype is in the list
df = df[~df['playtype'].isin(playtypes_to_remove)]

# Print results
rows_removed = initial_rows - len(df)
print(f"Rows removed: {rows_removed}")
print(f"New DataFrame shape: {df.shape}")

# Verify remaining playtypes
print("\nRemaining playtypes:")
print(df['playtype'].unique())

Rows removed: 8802
New DataFrame shape: (511882, 14)

Remaining playtypes:
['TO' 'ST' '2FGM' 'AS' '2FGA' 'O' '3FGA' 'D' 'CM' 'RV' 'IN' 'OUT' '3FGM'
 'FTM' 'TOUT' 'OF' 'FTA' 'AG' 'FV' 'CMT' 'CMU' 0 'TOUT_TV' 'C' 'B' 'CMD'
 'F']


In [6]:
df = df[df["playtype"] != 0]
df["markertime"] = df["markertime"].astype(str)
df["markertime"] = df["markertime"].str.replace("0 days 00:", "")

In [7]:
def fill_running_scores(df):
    """
    Forward fills basketball scores, converting rows with 0-0 to show running totals.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing columns 'points_a' and 'points_b'
    
    Returns:
    pandas.DataFrame: DataFrame with running totals for both teams
    """
    # Create a copy to avoid modifying the original DataFrame
    result_df = df.copy()
    
    # Convert 0s to NaN where both scores are 0
    mask = (result_df["points_a"] == 0) & (result_df["points_b"] == 0)
    result_df.loc[mask, ["points_a", "points_b"]] = np.nan
    
    # Forward fill the NaN values
    result_df["points_a"] = result_df["points_a"].fillna(method="ffill").fillna(0)
    result_df["points_b"] = result_df["points_b"].fillna(method="ffill").fillna(0)
    
    # Convert back to integers since scores should be whole numbers
    result_df["points_a"] = result_df["points_a"].astype(int)
    result_df["points_b"] = result_df["points_b"].astype(int)
    
    return result_df

df = fill_running_scores(df)

C:\Users\masii\AppData\Local\Temp\ipykernel_20252\3129420200.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result_df["points_a"] = result_df["points_a"].fillna(method="ffill").fillna(0)
C:\Users\masii\AppData\Local\Temp\ipykernel_20252\3129420200.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result_df["points_b"] = result_df["points_b"].fillna(method="ffill").fillna(0)


In [24]:
df["plus_minus"] = df["points_a"] - df["points_b"]

In [ ]:
def convert_time_to_seconds(time_str):
    """
    Convert time string from 'mm:ss' format to seconds
    
    Parameters:
    time_str: string in format 'mm:ss' (e.g., '10:00', '09:45', '00:30')
    
    Returns:
    int: total seconds
    """
    minutes, seconds = time_str.split(":")
    return int(minutes) * 60 + int(seconds)

# Apply the conversion
df["seconds"] = df["markertime"].apply(convert_time_to_seconds)


,unnamed: 0,numberofplay,codeteam,playtype,team,minute,markertime,points_a,points_b,playinfo,teama,teamb,game_id,quarter,plus_minus,seconds
0,1048,1,OLD,TO,EWE Baskets Oldenburg,1,09:46,0,0,Turnover (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,586
1,1049,2,BON,ST,Telekom Baskets Bonn,1,09:46,0,0,Steal (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,586
2,1050,3,BON,2FGM,Telekom Baskets Bonn,1,09:32,2,0,Two Pointer (1/1 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,2,572
3,1051,4,BON,AS,Telekom Baskets Bonn,1,09:29,2,0,Assist (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,2,569
4,1052,5,OLD,2FGM,EWE Baskets Oldenburg,1,09:22,2,2,Two Pointer (1/1 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562
5,1053,6,BON,2FGA,Telekom Baskets Bonn,1,09:22,2,2,Missed Two Pointer (1/2 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562
6,1054,7,BON,O,Telekom Baskets Bonn,1,09:22,2,2,Off Rebound (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562
7,1055,8,BON,3FGA,Telekom Baskets Bonn,1,09:22,2,2,Missed Three Pointer (0/1 - 0 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562
8,1056,9,BON,O,Telekom Baskets Bonn,1,09:22,2,2,Off Rebound (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562
9,1057,10,BON,3FGA,Telekom Baskets Bonn,1,09:22,2,2,Missed Three Pointer (0/1 - 0 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562


In [7]:
# First, let's see how many rows we have before cleaning
print(f"Original number of rows: {len(df)}")

# Create a copy of the df to work with
df_clean = df.copy()

# List of playtypes to remove
playtypes_to_remove = ["AS", "IN", "OUT", "TOUT", "TOUT_TV", "F", "AG", "FV"]

# Remove unnecessary playtypes
df_clean = df_clean[~df_clean["playtype"].isin(playtypes_to_remove)]

# Combine all technical fouls into CM
technical_fouls = ["CMT", "CMU", "C", "B", "CMD"]
df_clean["playtype"] = df_clean["playtype"].replace(technical_fouls, "CM")

# Let's see how many rows we have after cleaning
print(f"Number of rows after cleaning: {len(df_clean)}")

# Let's check the unique playtypes that remain to verify our cleaning
remaining_playtypes = sorted(df_clean["playtype"].unique())
print("\nRemaining playtypes:")
for pt in remaining_playtypes:
    count = len(df_clean[df_clean["playtype"] == pt])
    print(f"{pt}: {count} occurrences")

df = df_clean

Original number of rows: 511880
Number of rows after cleaning: 351093

Remaining playtypes:
2FGA: 35889 occurrences
2FGM: 39719 occurrences
3FGA: 29623 occurrences
3FGM: 16346 occurrences
CM: 39072 occurrences
D: 46986 occurrences
FTA: 9546 occurrences
FTM: 28467 occurrences
O: 20326 occurrences
OF: 3303 occurrences
RV: 41675 occurrences
ST: 13332 occurrences
TO: 26809 occurrences


In [26]:
# Create a new dataframe to adjust "numberofplay"
new_df = df.copy()

# Update "numberofplay" within each "game_id"
new_df['numberofplay'] = new_df.groupby('game_id').cumcount() + 1
df = new_df

In [16]:
# Define the playtypes that stop the correction
stop_playtypes = ["2FGM", "3FGM", "FTM"]

# Create a function to correct points within each game_id
def correct_points(group):
    # Initialize the points
    points_a, points_b = 0, 0
    
    # Iterate over rows to update points
    for idx, row in group.iterrows():
        # Update points to 0 until a stop playtype is found
        if row["playtype"] in stop_playtypes:
            break
        group.at[idx, "points_a"] = points_a
        group.at[idx, "points_b"] = points_b
    
    return group

# Apply the correction function to each game_id group
new_df = new_df.groupby("game_id", group_keys=False).apply(correct_points)

# Display the updated dataframe
df = new_df

C:\Users\masii\AppData\Local\Temp\ipykernel_13040\573329788.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df = new_df.groupby("game_id", group_keys=False).apply(correct_points)


In [22]:
# Define the required quarters
required_quarters = {1, 2, 3, 4}

# Identify the game_ids that have all four quarters
valid_game_ids = (
    new_df.groupby("game_id")["quarter"]
    .apply(lambda x: required_quarters.issubset(set(x)))
)

# Filter to keep only rows with valid game_ids
filtered_df = new_df[new_df["game_id"].isin(valid_game_ids[valid_game_ids].index)]

# Display the filtered dataframe
filtered_df.head()

# Count unique game_id values before and after filtering
original_game_ids = new_df["game_id"].nunique()
filtered_game_ids = filtered_df["game_id"].nunique()

# Calculate the number of dropped game_ids
dropped_game_ids = original_game_ids - filtered_game_ids

print(f"Number of game_id dropped: {dropped_game_ids}")

df = filtered_df

Number of game_id dropped: 3


In [25]:
# Filter out rows where playtype is "CM" or "OF"
df = df[~df["playtype"].isin(["CM", "OF"])]

df["playtype"].unique()

array(['TO', 'ST', '2FGM', '2FGA', 'O', '3FGA', 'D', 'RV', '3FGM', 'FTM',
       'FTA'], dtype=object)

In [30]:
# Create a new column 'possession'
df['possession'] = df.apply(
    lambda row: 1 if row['team'] == row['teama'] else 2,
    axis=1
)

# Display the updated dataframe to verify
df.head()

,unnamed: 0,numberofplay,codeteam,playtype,team,minute,markertime,points_a,points_b,playinfo,teama,teamb,game_id,quarter,plus_minus,seconds,possession
0,1048,1,OLD,TO,EWE Baskets Oldenburg,1,09:46,0,0,Turnover (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,586,2
1,1049,2,BON,ST,Telekom Baskets Bonn,1,09:46,0,0,Steal (1),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,586,1
2,1050,3,BON,2FGM,Telekom Baskets Bonn,1,09:32,2,0,Two Pointer (1/1 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,2,572,1
4,1052,4,OLD,2FGM,EWE Baskets Oldenburg,1,09:22,2,2,Two Pointer (1/1 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562,2
5,1053,5,BON,2FGA,Telekom Baskets Bonn,1,09:22,2,2,Missed Two Pointer (1/2 - 2 pt),Telekom Baskets Bonn,EWE Baskets Oldenburg,2015_001,1,0,562,1


In [36]:
# Columns to drop
columns_to_drop = ['codeteam', 'teama', 'teamb', 'team']

# Drop the specified columns
df_dropped = df.drop(columns=columns_to_drop)

# Display the updated dataframe
df_dropped.head()
df = df_dropped

In [37]:
# ¡¡¡¡¡¡¡¡!!!!!!!!! Every row above this one MEANS that the df here saved already has them implemented in the data wrangling!!!
df.to_csv('eurocup.csv', index=False)

In [38]:
df.head()

,unnamed: 0,numberofplay,playtype,minute,markertime,points_a,points_b,playinfo,game_id,quarter,plus_minus,seconds,possession
0,1048,1,TO,1,09:46,0,0,Turnover (1),2015_001,1,0,586,2
1,1049,2,ST,1,09:46,0,0,Steal (1),2015_001,1,0,586,1
2,1050,3,2FGM,1,09:32,2,0,Two Pointer (1/1 - 2 pt),2015_001,1,2,572,1
4,1052,4,2FGM,1,09:22,2,2,Two Pointer (1/1 - 2 pt),2015_001,1,0,562,2
5,1053,5,2FGA,1,09:22,2,2,Missed Two Pointer (1/2 - 2 pt),2015_001,1,0,562,1
